# Polynomial Regression

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import minterpy as mp

Describe the problem of regression (modeling on scattered data):

- Example case used
- Steps to come up with the regression
- The required transformation
- Evaluation on testing points
- Some validations

Use common sense how to section the tutorial, we can discuss it later if reorganization is needed.

Put any required functions related to regressions not yet merged to the `dev` branch locally here in the notebook (I guess based on our discussion it should not be that large).

Before you push the notebook, don't forget to clear all the outputs